# Полный цикл подготовки TVR (v2 - с заполненными данными)

Этот ноутбук собирает все шаги: от просмотра исходного TVR до формирования шаблона по маске и генерации итоговой таблицы.

**Отличие v2:** Маска создается с данными из исходного TVR, маркер override - символ "#"

In [20]:
import sys
from pathlib import Path

project_root = Path.cwd().resolve()
if not (project_root / 'src').exists():
    project_root = project_root.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
project_root

WindowsPath('C:/Users/user/Documents/piranha/constructor_TVR')

## Шаг 1. Загрузка TVR и выгрузка в Excel

1. Читаем исходный TVR-файл.
2. Сохраняем его в Excel, чтобы можно было визуально осмотреть все связи и stroka.
3. Смотрим первые строки для быстрой проверки.

**Что можно менять:** путь к `source_tvr_path` и имя сохранённого Excel.

In [21]:
from tvr_df import TVR_asis
import pandas as pd

source_tvr_path = Path(project_root) / 'Default.tvr2'  # при необходимости укажите другой исходник
excel_snapshot_path = Path(project_root) / 'docs' / 'default_snapshot.xlsx'

tvr_full = TVR_asis(source_tvr_path)
tvr_full.to_excel(excel_snapshot_path, index=False)

print(f'Excel со всеми строками сохранён: {excel_snapshot_path}')
# tvr_full.head(10)

Excel со всеми строками сохранён: C:\Users\user\Documents\piranha\constructor_TVR\docs\default_snapshot.xlsx


## Шаг 2. Выбор базовых строк для парсинга


In [22]:
# Пример: вручную выбраны stroka для разбора (long и short)
selected_strokas = [3468, 3469]
selected_strokas

[3468, 3469]

## Шаг 3. Переориентация выбранного шаблона

Загружаем структуру только по нужным stroka, выводим её в текстовом виде, вручную задаём новые смещения и компилируем результат.

**Что можно менять:**
- `target_base_stroka` — абсолютный номер, куда ставим базовую строку.
- содержимое `edited_text` — новые offsets для нужных узлов.

In [23]:
from src.tvr_service.generator import build_layout_from_source
from src.tvr_service.generator.layout import LayoutEdits, compile_layout

layout = build_layout_from_source(str(source_tvr_path), selected_strokas)
print(f'Основная база (до переориентации): {layout.primary_base_stroka}')

Основная база (до переориентации): 3468


### Структура в текстовом виде

Скопируйте блок ниже и правьте offsets вручную (значение после двоеточия).

In [24]:
# Генерация текстовой структуры с использованием order
lines = []
for entry in layout.entries:
    label = entry.label.replace(' ', '_')
    lines.append(f"{entry.order}: {label}: {entry.relative_offset}:")
layout_text = '\n'.join(lines)
print(layout_text)


1: base_long: 0:
2: filter_1_long_&_short: 2:
3: filter_2_long_&_short: 3:
4: filter_3_long_&_short: 5:
5: filter_4_long_&_short: 6:
6: base_short: 1:


### Редактируем смещения вручную

Ниже пример со смещениями: после второго двоеточия указано новое значение.
При необходимости замените на свои цифры.

In [25]:
edited_text = '''
1: base_long: 0: 0
2: filter_1_long_&_short: 2: 2
3: filter_2_long_&_short: 3: 3
4: filter_3_long_&_short: 5: 4
5: filter_4_long_&_short: 6: 5
6: base_short: 1: 1
'''

target_base_stroka = 1000  # куда переносим базовую строку



def parse_simple_overrides(layout, text):
    overrides = {}
    
    # Создаем словарь order -> entry
    order_to_entry = {entry.order: entry for entry in layout.entries}
    
    for line in text.strip().splitlines():
        if ':' not in line:
            continue
        parts = [chunk.strip() for chunk in line.split(':')]
        if len(parts) < 4:  # Теперь нужно 4 части
            continue
        
        order_str, name, old_offset, new_offset = parts[0], parts[1], parts[2], parts[3]
        if new_offset == '':
            continue
        
        try:
            order = int(order_str)
            new_offset = int(new_offset)
        except ValueError:
            print(f"Ошибка парсинга строки: {line}")
            continue
        
        if order in order_to_entry:
            entry = order_to_entry[order]
            overrides[entry.original_stroka] = new_offset
        else:
            print(f"Не найдена запись с order={order}")
    
    return LayoutEdits(relative_overrides=overrides)

edits = parse_simple_overrides(layout, edited_text)
compiled = compile_layout(layout, edits, base_assignment={layout.primary_base_stroka: target_base_stroka})
compiled

CompiledLayout(base_assignment={3468: 1000, 3469: 1001}, stroka_overrides={3470: 1002, 3471: 1003, 3473: 1004, 3474: 1005})

## Шаг 4. Заготовка маски в Excel с данными из исходного TVR

**ВАЖНО (v2):** Теперь маска заполняется данными из спарсенной стратегии!

На основе пересобранного шаблона формируем таблицу с:
- `row_alias` - алиасы строк из шаблона
- `stroka` - новые номера строк
- **данные из исходного TVR** - копируются из спарсенных строк

Оператор отмечает нужные ячейки символом **"#"** для переопределения в config_table.

Файл включает:
- блок шаблона с данными из исходного TVR;
- три пустые строки-разделителя;
- пример строк из исходного робота (для сверки).

In [26]:
# НОВЫЙ МЕТОД: Используем встроенную функцию для создания маски с относительными ссылками
from src.tvr_service.generator import build_mask_dataframe_from_layout

# Создаем маску с автоматической конвертацией ссылок в относительный формат
mask_template_df = build_mask_dataframe_from_layout(
    layout=layout,
    compiled=compiled,
    source_dataframe=tvr_full,
    target_base_stroka=target_base_stroka,
)

print("✓ Маска создана с относительными ссылками на фильтры (begin+X)")
print(f"  Базовая строка для расчета: {layout.primary_base_stroka}")

# Полный перечень колонок TVR
tvr_columns = list(tvr_full.columns)
mask_columns = ['row_alias', *tvr_columns]

# Три пустые строки-разделителя
separator_df = pd.DataFrame([{col: pd.NA for col in mask_columns} for _ in range(3)])

# Пример строк из исходного робота для наглядности (для сверки)
sample_strokas = sorted([entry.original_stroka for entry in layout.entries])
sample_df = tvr_full[tvr_full['stroka'].isin(sample_strokas)].copy()
sample_df.insert(0, 'row_alias', [f'sample_{int(st)}' for st in sample_df['stroka']])
sample_df = sample_df.reindex(columns=mask_columns)

mask_output_df = pd.concat([mask_template_df, separator_df, sample_df], ignore_index=True, sort=False)
mask_template_path = Path(project_root) / 'docs' / 'mask_template_v2.xlsx'
mask_output_df.to_excel(mask_template_path, index=False)

print(f'\nШаблон маски с данными сохранён: {mask_template_path}')
print(f'Теперь откройте файл и замените нужные ячейки на "#" для переопределения')
print(f'Сохраните результат как mask.xlsx')
mask_output_df.head(15)


✓ Маска создана с относительными ссылками на фильтры (begin+X)
  Базовая строка для расчета: 3468

Шаблон маски с данными сохранён: C:\Users\user\Documents\piranha\constructor_TVR\docs\mask_template_v2.xlsx
Теперь откройте файл и замените нужные ячейки на "#" для переопределения
Сохраните результат как mask.xlsx


,row_alias,stroka,Start,Kill all,Out only,InL1,InL2,OutL1,OutL2,Pos,...,T.In,T.Out,NotSet,FrId,MoveN,secIn,secOut,XN,MP,xx
0,1_base_long,1000,True,NaN,NaN,begin+2,NaN,begin+2,NaN,10;-1;,...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,NaN,5,3/3,NaN,NaN,0.005,NaN,NaN
1,2_filter_1_long_&_short,1002,NaN,NaN,NaN,begin+3,NaN,begin+3,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
2,3_filter_2_long_&_short,1003,NaN,NaN,NaN,begin+4,NaN,begin+4,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4_filter_3_long_&_short,1004,NaN,NaN,NaN,begin+5,NaN,begin+5,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5_filter_4_long_&_short,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6_base_short,1001,True,NaN,NaN,begin+2,NaN,begin+2,NaN,NaN,...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,NaN,NaN,3/3,NaN,NaN,0.005,NaN,NaN
6,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,sample_3468,3468,True,NaN,NaN,3470,NaN,3470,NaN,10;-1;,...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,NaN,5,3/3,NaN,NaN,0.005,NaN,NaN


### Проверка конвертации ссылок

Давайте проверим, что ссылки в столбцах фильтров (InL1, InL2, OutL1, OutL2) были корректно конвертированы в относительный формат:


### 🎯 Важно: Пересчет ссылок при пересборке

Обратите внимание, что функция **автоматически пересчитывает ссылки** при изменении offset!

Например, если:
- Фильтр был на `offset=10` (stroka 4990)
- Мы переместили его на `offset=2` (новая stroka 1002)
- Ссылка на него была `InL1: 4990` → станет `InL1: begin+2` (а не `begin+10`)

Это критически важно для корректной работы при пересборке layout!


In [27]:
# Показываем столбцы с фильтрами для первых нескольких строк
filter_columns = ['row_alias', 'stroka', 'InL1', 'InL2', 'OutL1', 'OutL2']
available_filter_cols = [col for col in filter_columns if col in mask_template_df.columns]

print("Проверка конвертации ссылок на фильтры:")
print("=" * 80)
print(f"\nБазовая строка для относительных ссылок: {layout.primary_base_stroka}")
print("\nСсылки в относительном формате (begin+X):")
print("-" * 80)

display_df = mask_template_df[available_filter_cols].head(10)
# Показываем только строки, где есть хотя бы одна ссылка
has_refs = display_df[available_filter_cols[2:]].notna().any(axis=1)
display_df = display_df[has_refs]

if not display_df.empty:
    print(display_df.to_string(index=False))
    print("\n✓ Ссылки успешно конвертированы в относительный формат!")
    print("  Теперь при генерации с разными 'start' ссылки будут автоматически")
    print("  пересчитываться относительно новой базовой строки.")
else:
    print("Нет строк с ссылками на фильтры в первых 10 записях")


Проверка конвертации ссылок на фильтры:

Базовая строка для относительных ссылок: 3468

Ссылки в относительном формате (begin+X):
--------------------------------------------------------------------------------
              row_alias  stroka    InL1  InL2   OutL1  OutL2
            1_base_long    1000 begin+2   NaN begin+2    NaN
2_filter_1_long_&_short    1002 begin+3   NaN begin+3    NaN
3_filter_2_long_&_short    1003 begin+4   NaN begin+4    NaN
4_filter_3_long_&_short    1004 begin+5   NaN begin+5    NaN
           6_base_short    1001 begin+2   NaN begin+2    NaN

✓ Ссылки успешно конвертированы в относительный формат!
  Теперь при генерации с разными 'start' ссылки будут автоматически
  пересчитываться относительно новой базовой строки.


### Загрузка отредактированной маски

После того как вы:
1. Открыли `mask_template_v2.xlsx`
2. Заменили нужные значения на символ **"#"** (те ячейки, которые будут браться из config_table)
3. Сохранили как `mask.xlsx`

Загружаем маску с маркером "#":


In [28]:
mask_path = Path(project_root) / 'mask.xlsx'
mask_df = pd.read_excel(mask_path)
print('Текущая маска:')
mask_df

Текущая маска:


,row_alias,stroka,Start,Kill all,Out only,InL1,InL2,OutL1,OutL2,Pos,...,T.In,T.Out,NotSet,FrId,MoveN,secIn,secOut,XN,MP,xx
0,1_base_long,1000,True,NaN,NaN,begin+2,NaN,begin+2,NaN,NaN,...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,NaN,5.0,3/3,NaN,NaN,0.005,NaN,NaN
1,6_base_short,1001,True,NaN,NaN,begin+2,NaN,begin+2,NaN,NaN,...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,NaN,NaN,3/3,NaN,NaN,0.005,NaN,NaN
2,2_filter_1_long_&_short,1002,NaN,NaN,NaN,begin+3,NaN,begin+3,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3_filter_2_long_&_short,1003,NaN,NaN,NaN,begin+4,NaN,begin+4,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4_filter_3_long_&_short,1004,NaN,NaN,NaN,begin+5,NaN,begin+5,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5_filter_4_long_&_short,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Построение шаблона из маски с маркером "#"

Теперь используем символ "#" как маркер для override:


In [29]:
# Проверка на дубликаты в row_alias (опционально)
mask_unique = mask_df['row_alias'].dropna().duplicated()
if mask_unique.any():
    print("ВНИМАНИЕ: Найдены дубликаты в row_alias!")
    print(mask_df[mask_unique])
else:
    print("✓ Дубликатов в row_alias не найдено")

✓ Дубликатов в row_alias не найдено


In [30]:
from src.tvr_service.templates import build_template_from_mask_file

# ВАЖНО: build_template_from_mask_file автоматически конвертирует относительные ссылки
# (begin+X) обратно в абсолютные при генерации TVR!
# 
# По умолчанию convert_references_to_relative=True, что означает:
# 1. При ЧТЕНИИ маски - относительные ссылки НЕ конвертируются (остаются как есть)
# 2. При ГЕНЕРАЦИИ из шаблона - относительные ссылки конвертируются в абсолютные
#    с учетом значения 'start' для каждой стратегии

mask_result = build_template_from_mask_file(
    mask_path,
    row_alias_column='row_alias',
    marker='#',  # ИЗМЕНЕНО: используем '#' вместо 1
)

print('✓ Шаблон создан с поддержкой относительных ссылок')
print('\nКолонки для конфигурации (помеченные "#"):')
print(mask_result.override_columns)
print(f'\nВсего override колонок: {len(mask_result.override_columns)}')

✓ Шаблон создан с поддержкой относительных ссылок

Колонки для конфигурации (помеченные "#"):
['base_long_SM', 'base_long_Sec_0', 'base_long_V_0', 'base_long_W_1', 'base_short_SM', 'base_short_Sec_0', 'base_short_V_0', 'base_short_W_1', 'filter_1_long_short_Sec_0', 'filter_1_long_short_W_1', 'filter_2_long_short_Sec_0', 'filter_2_long_short_Sec_1', 'filter_3_long_short_Sec_0', 'filter_3_long_short_Sec_1', 'filter_4_long_short_Sec_0', 'filter_4_long_short_W_1']

Всего override колонок: 16


## Шаг 5. Формирование config_table

Создаем основу для config_table на основе портфолио и маски:
1. Загружаем портфолио из porfolio_base.csv
2. Получаем override_columns из маски
3. Рассчитываем позиции для стратегий
4. Создаем config_table с нужными колонками
5. Сохраняем результат в Excel файл


In [31]:
# 1. Загружаем портфолио
print("1. Загружаем портфолио...")
port_base = pd.read_csv('porfolio_base.csv', header=0, index_col=0)
print(f"Загружено {len(port_base)} стратегий")
print(f"Колонки портфолио: {list(port_base.columns)}")
port_base.head()


1. Загружаем портфолио...
Загружено 43 стратегий
Колонки портфолио: ['strategy_id', 'Sec_0', 'sec_0_price', 'sec_0_GO', 'sec_1', 'W_0', 'SM', 'SM_sh', 'sec_1_GO', 'param_num', 'V_0']


,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,SM_sh,sec_1_GO,param_num,V_0
0,AFZ5_1,AFZ5,5655.0,1467.66,MMZ5,0.204,1;1;0.1;0.7,4;7;0.2;0.8,3288.25,1,7
1,AKZ5_1,AKZ5,13946.0,3686.51,MMZ5,0.504,1;2;0.1;0.7,4;18;0.2;0.8,3288.25,1,3
2,ALZ5_1,ALZ5,4473.0,1174.04,MMZ5,0.161,1;1;0.1;0.7,4;5;0.2;0.8,3288.25,1,9
3,ASZ5_1,ASZ5,320.0,108.39,MMZ5,0.011,1;1;0.1;0.7,4;1;0.2;0.8,3288.25,1,107
4,BNZ5_1,BNZ5,1566.0,535.91,MMZ5,0.056,1;1;0.1;0.7,4;2;0.2;0.8,3288.25,1,21


In [32]:
from src.tvr_service.templates import build_template_from_mask_file

mask_result = build_template_from_mask_file(
    mask_path,
    row_alias_column='row_alias',
    marker='#',
)

print('Колонки для конфигурации:')
mask_result.override_columns

Колонки для конфигурации:


['base_long_SM',
 'base_long_Sec_0',
 'base_long_V_0',
 'base_long_W_1',
 'base_short_SM',
 'base_short_Sec_0',
 'base_short_V_0',
 'base_short_W_1',
 'filter_1_long_short_Sec_0',
 'filter_1_long_short_W_1',
 'filter_2_long_short_Sec_0',
 'filter_2_long_short_Sec_1',
 'filter_3_long_short_Sec_0',
 'filter_3_long_short_Sec_1',
 'filter_4_long_short_Sec_0',
 'filter_4_long_short_W_1']

In [33]:
# 2. Получаем override_columns из маски
print("\n2. Получаем override_columns из маски...")
print(f"Override columns: {mask_result.override_columns}")
print(f"Количество override columns: {len(mask_result.override_columns)}")



2. Получаем override_columns из маски...
Override columns: ['base_long_SM', 'base_long_Sec_0', 'base_long_V_0', 'base_long_W_1', 'base_short_SM', 'base_short_Sec_0', 'base_short_V_0', 'base_short_W_1', 'filter_1_long_short_Sec_0', 'filter_1_long_short_W_1', 'filter_2_long_short_Sec_0', 'filter_2_long_short_Sec_1', 'filter_3_long_short_Sec_0', 'filter_3_long_short_Sec_1', 'filter_4_long_short_Sec_0', 'filter_4_long_short_W_1']
Количество override columns: 16


In [34]:
from strategy_position_calculator import calculate_strategy_positions
# 3. Рассчитываем позиции для стратегий
print("\n3. Рассчитываем позиции...")
positions = calculate_strategy_positions(
    start_number=530,
    rows_between=4,
    num_strategies=len(port_base),
    mask_df=mask_df
)
print(f"Позиции: {positions}")
print(f"Шаг между позициями: {positions[1] - positions[0] if len(positions) > 1 else 'N/A'}")



3. Рассчитываем позиции...
Позиции: [530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950]
Шаг между позициями: 10


In [35]:
# 4. Создаем config_table
print("\n4. Создаем config_table...")

# Копируем все данные из port_base
config_table = port_base.copy()

# Добавляем start_column с позициями
config_table['start_column'] = positions

# Добавляем пустые колонки из override_columns
for col in mask_result.override_columns:
    config_table[col] = ''

print(f"Config table создан с {len(config_table)} строками и {len(config_table.columns)} колонками")
print(f"Колонки: {list(config_table.columns)}")
config_table.head()



4. Создаем config_table...
Config table создан с 43 строками и 28 колонками
Колонки: ['strategy_id', 'Sec_0', 'sec_0_price', 'sec_0_GO', 'sec_1', 'W_0', 'SM', 'SM_sh', 'sec_1_GO', 'param_num', 'V_0', 'start_column', 'base_long_SM', 'base_long_Sec_0', 'base_long_V_0', 'base_long_W_1', 'base_short_SM', 'base_short_Sec_0', 'base_short_V_0', 'base_short_W_1', 'filter_1_long_short_Sec_0', 'filter_1_long_short_W_1', 'filter_2_long_short_Sec_0', 'filter_2_long_short_Sec_1', 'filter_3_long_short_Sec_0', 'filter_3_long_short_Sec_1', 'filter_4_long_short_Sec_0', 'filter_4_long_short_W_1']


,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,SM_sh,sec_1_GO,param_num,...,base_short_V_0,base_short_W_1,filter_1_long_short_Sec_0,filter_1_long_short_W_1,filter_2_long_short_Sec_0,filter_2_long_short_Sec_1,filter_3_long_short_Sec_0,filter_3_long_short_Sec_1,filter_4_long_short_Sec_0,filter_4_long_short_W_1
0,AFZ5_1,AFZ5,5655.0,1467.66,MMZ5,0.204,1;1;0.1;0.7,4;7;0.2;0.8,3288.25,1,...,,,,,,,,,,
1,AKZ5_1,AKZ5,13946.0,3686.51,MMZ5,0.504,1;2;0.1;0.7,4;18;0.2;0.8,3288.25,1,...,,,,,,,,,,
2,ALZ5_1,ALZ5,4473.0,1174.04,MMZ5,0.161,1;1;0.1;0.7,4;5;0.2;0.8,3288.25,1,...,,,,,,,,,,
3,ASZ5_1,ASZ5,320.0,108.39,MMZ5,0.011,1;1;0.1;0.7,4;1;0.2;0.8,3288.25,1,...,,,,,,,,,,
4,BNZ5_1,BNZ5,1566.0,535.91,MMZ5,0.056,1;1;0.1;0.7,4;2;0.2;0.8,3288.25,1,...,,,,,,,,,,


In [36]:
# 5. Сохраняем config_table в Excel
print("\n5. Сохраняем config_table в Excel...")
config_table_path = Path(project_root) / 'docs' / 'config_table_v2.xlsx'
config_table.to_excel(config_table_path, index=True)
print(f"Config table сохранен: {config_table_path}")

# Показываем информацию о созданной таблице
print(f"\nРезультат:")
print(f"Файл: {config_table_path}")
print(f"Размер: {config_table.shape}")
print(f"Информация о start_column:")
print(f"Значения: {config_table['start_column'].tolist()}")
print(f"Уникальных значений: {config_table['start_column'].nunique()}")

# Проверяем override columns
override_cols = [col for col in config_table.columns if col not in ['strategy_id', 'Sec_0', 'sec_0_price', 'sec_0_GO', 'sec_1', 'W_0', 'SM', 'param_num', 'V_0', 'start_column']]
print(f"\nИнформация о override columns:")
print(f"Количество override columns: {len(override_cols)}")
print(f"Все пустые: {config_table[override_cols].isna().all().all()}")



5. Сохраняем config_table в Excel...
Config table сохранен: C:\Users\user\Documents\piranha\constructor_TVR\docs\config_table_v2.xlsx

Результат:
Файл: C:\Users\user\Documents\piranha\constructor_TVR\docs\config_table_v2.xlsx
Размер: (43, 28)
Информация о start_column:
Значения: [530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950]
Уникальных значений: 43

Информация о override columns:
Количество override columns: 18
Все пустые: False


## Шаг 6. Загрузка и редактирование config_table

Теперь вы можете:
1. Открыть созданный файл `docs/config_table_v2.xlsx` в Excel
2. Заполнить override columns нужными значениями (только те, что были помечены "#")
3. Сохранить файл
4. Загрузить обновленный файл обратно в ноутбук


In [37]:
# Загружаем обновленный config_table (после ручного редактирования)
print("Загружаем обновленный config_table...")
config_table_updated = pd.read_excel(config_table_path, index_col=0)
print(f"Загружен config_table с {len(config_table_updated)} строками и {len(config_table_updated.columns)} колонками")

# Показываем первые несколько строк
print("\nПервые 3 строки обновленного config_table:")
config_table_updated.head(3)


Загружаем обновленный config_table...
Загружен config_table с 43 строками и 28 колонками

Первые 3 строки обновленного config_table:


,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,SM_sh,sec_1_GO,param_num,...,base_short_V_0,base_short_W_1,filter_1_long_short_Sec_0,filter_1_long_short_W_1,filter_2_long_short_Sec_0,filter_2_long_short_Sec_1,filter_3_long_short_Sec_0,filter_3_long_short_Sec_1,filter_4_long_short_Sec_0,filter_4_long_short_W_1
0,AFZ5_1,AFZ5,5655,1467.66,MMZ5,0.204,1;1;0.1;0.7,4;7;0.2;0.8,3288.25,1,...,7,2.04,AFZ5,2.04,AFZ5,AFZ5,AFZ5,AFZ5,AFZ5,4.08
1,AKZ5_1,AKZ5,13946,3686.51,MMZ5,0.504,1;2;0.1;0.7,4;18;0.2;0.8,3288.25,1,...,3,5.04,AKZ5,5.04,AKZ5,AKZ5,AKZ5,AKZ5,AKZ5,10.08
2,ALZ5_1,ALZ5,4473,1174.04,MMZ5,0.161,1;1;0.1;0.7,4;5;0.2;0.8,3288.25,1,...,9,1.61,ALZ5,1.61,ALZ5,ALZ5,ALZ5,ALZ5,ALZ5,3.22


## Шаг 7. Генерация TVR с помощью StrategyGenerator

Используем загруженный config_table для генерации TVR файлов с помощью StrategyGenerator.

**Важно:** Теперь TVR будет содержать данные из исходного TVR (defaults) + данные из config_table (только для ячеек с "#").


In [38]:
# Создаем StrategyGenerator
from src.tvr_service.generator import StrategyGenerator

generator = StrategyGenerator(
    mask_result.template,
    start_column='start_column',  # используем нашу колонку с позициями
    strategy_column='strategy_id',
    sec_column=None,
)

print("StrategyGenerator создан успешно")
print(f"Template: {generator.template.name}")
print(f"Start column: {generator.start_column}")
print(f"Strategy column: {generator.strategy_column}")


StrategyGenerator создан успешно
Template: mask
Start column: start_column
Strategy column: strategy_id


In [39]:
# Генерируем TVR для всех стратегий
print("Генерируем TVR для всех стратегий...")
tvr_result = generator.generate(config_table_updated, blank_rows_between=1)

print(f"Сгенерировано {len(tvr_result)} строк")

# Показываем первые несколько строк результата
print("\nПервые 10 строк сгенерированного TVR:")
tvr_result.head(10)


Генерируем TVR для всех стратегий...
Сгенерировано 301 строк

Первые 10 строк сгенерированного TVR:


,stroka,Start,Kill all,Out only,InL1,InL2,OutL1,OutL2,Pos,Limit,...,T.In,T.Out,NotSet,FrId,MoveN,secIn,secOut,XN,MP,xx
0,530,True,<NA>,<NA>,532,<NA>,532,<NA>,<NA>,<NA>,...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,<NA>,5.0,3/3,<NA>,<NA>,0.005,<NA>,<NA>
1,531,True,<NA>,<NA>,532,<NA>,532,<NA>,<NA>,<NA>,...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,<NA>,<NA>,3/3,<NA>,<NA>,0.005,<NA>,<NA>
2,532,<NA>,<NA>,<NA>,533,<NA>,533,<NA>,<NA>,<NA>,...,09:00:00-23:50:00,09:00:00-23:50:00,<NA>,5.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,533,<NA>,<NA>,<NA>,534,<NA>,534,<NA>,<NA>,<NA>,...,09:00:00-23:50:00,09:00:00-23:50:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,534,<NA>,<NA>,<NA>,535,<NA>,535,<NA>,<NA>,<NA>,...,09:00:00-23:50:00,09:00:00-23:50:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,535,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,09:00:00-23:50:00,09:00:00-23:50:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,540,True,<NA>,<NA>,542,<NA>,542,<NA>,<NA>,<NA>,...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,<NA>,5.0,3/3,<NA>,<NA>,0.005,<NA>,<NA>
8,541,True,<NA>,<NA>,542,<NA>,542,<NA>,<NA>,<NA>,...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,09:00:20-09:20:00 09:21:00-10:20:00 10:21:00-1...,<NA>,<NA>,3/3,<NA>,<NA>,0.005,<NA>,<NA>
9,542,<NA>,<NA>,<NA>,543,<NA>,543,<NA>,<NA>,<NA>,...,09:00:00-23:50:00,09:00:00-23:50:00,<NA>,5.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [40]:
# Сохраняем результат в Excel для проверки
tvr_output_path = Path(project_root) / 'docs' / 'generated_tvr_result_v2.xlsx'
tvr_result.to_excel(tvr_output_path, index=False)
print(f"Результат сохранен в Excel: {tvr_output_path}")

# Показываем статистику
if 'strategy_id' in tvr_result.columns:
    strategy_counts = tvr_result['strategy_id'].value_counts()
    print(f"\nКоличество строк по стратегиям:")
    print(strategy_counts.head(10))

# Показываем диапазон позиций
print(f"\nДиапазон позиций:")
print(f"Минимальная позиция: {tvr_result['stroka'].min()}")
print(f"Максимальная позиция: {tvr_result['stroka'].max()}")
print(f"Уникальных позиций: {tvr_result['stroka'].nunique()}")


Результат сохранен в Excel: C:\Users\user\Documents\piranha\constructor_TVR\docs\generated_tvr_result_v2.xlsx

Диапазон позиций:
Минимальная позиция: 530
Максимальная позиция: 955
Уникальных позиций: 258


## Шаг 8. Сохранение в TVR2 формат

Сохраняем результат в формате TVR2:


In [ ]:
# import importlib
# import sys

# # Перезагружаем модуль tvr_io
# importlib.reload(sys.modules['src.tvr_service.io.tvr_io'])

# # Перезагружаем весь пакет io
# importlib.reload(sys.modules['src.tvr_service.io'])

# # Перезагружаем импорт
# from src.tvr_service.io import write_tvr2

In [41]:
# Сохраняем в TVR2 формат
from src.tvr_service.io import write_tvr2


tvr2_output_path = Path(project_root) / 'docs' / 'generated_strategies_v2.tvr2'
write_tvr2(tvr_result, tvr2_output_path)
print(f"TVR2 файл сохранен: {tvr2_output_path}")

# Проверяем, что файл создался
if tvr2_output_path.exists():
    print(f"Размер файла: {tvr2_output_path.stat().st_size} байт")
else:
    print("Ошибка: файл не создан")


TVR2 файл сохранен: C:\Users\user\Documents\piranha\constructor_TVR\docs\generated_strategies_v2.tvr2
Размер файла: 116588 байт


## Резюме

Весь процесс выполнен успешно:

### Отличия версии v2:
1. ✅ **Маска заполнена данными из исходного TVR** - не пустая!
2. ✅ **Маркер override - символ "#"** вместо 1
3. ✅ **Правильное сопоставление** row_alias ↔ original_stroka

### Этапы:
1. ✅ **Загружен исходный TVR** и спарсены нужные строки
2. ✅ **Создан layout** с переопределенными offset
3. ✅ **Создана маска С ДАННЫМИ** из исходного TVR
4. ✅ **Помечены "#" ячейки** для override
5. ✅ **Создан config_table** с override колонками
6. ✅ **Сгенерирован TVR** с помощью StrategyGenerator
7. ✅ **Сохранен результат** в Excel и TVR2 форматах

**Созданные файлы:**
- `docs/mask_template_v2.xlsx` - шаблон маски с данными
- `mask.xlsx` - отредактированная маска с "#"
- `docs/config_table_v2.xlsx` - конфигурация стратегий
- `docs/generated_tvr_result_v2.xlsx` - результат в Excel
- `docs/generated_strategies_v2.tvr2` - результат в TVR2 формате
